In [1]:
import numpy as np;

In [2]:
def init_params():
    W1 = np.random.normal(size=(10, 784)) * np.sqrt(1./(784))
    b1 = np.zeros((10, 1))
    W2 = np.random.normal(size=(10, 10)) * np.sqrt(1./10)
    b2 = np.zeros((10, 1))
    return W1, b1, W2, b2

In [3]:
def ReLU(Z):
    return np.maximum(Z, 0)

In [11]:
def softmax(Z):
    expZ = np.exp(Z - np.max(Z))  # prevent overflow by subtracting max
    return expZ / expZ.sum(axis=0)

In [10]:
def forward_propagation(X, W1, b1, W2, b2):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

In [7]:
def one_hot_encoding(Y):
    y_one_hot = np.zeros((Y.max+1, Y.size))
    y_one_hot[Y, np.arange(Y.size)] = 1
    return y_one_hot

In [5]:
def derivative_ReLU(z):
    return (z > 0).astype(float)

In [ ]:
def backward_propagation(X, Y, A1, A2, W2, Z1, m):
    one_hot_Y = one_hot_encoding(Y)
    dZ2 = (A2 - one_hot_Y) #10,m //drop constant
    dW2 = 1/m * (dZ2.dot(A1.T)) # 10 , 10
    db2 = 1/m * np.sum(dZ2, 1, axis=1, keepdims=True) # 10, 1
    dZ1 = W2.T.dot(dZ2)*derivative_ReLU(Z1) # 10, m
    dW1 = 1/m * (dZ1.dot(X.T)) #10, 784
    db1 = 1/m * np.sum(dZ1, 1, axis=1, keepdims=True) # 10, 1

    return dW1, db1, dW2, db2

In [ ]:
def update_params(learning_rate, W1, b1, W2, b2, dW1, db1, dW2, db2):
    W1 -= learning_rate * dW1
    b1 -= learning_rate * np.reshape(db1, (10,1))
    W2 -= learning_rate * dW2
    b2 -= learning_rate * np.reshape(db2, (10,1))

    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

In [ ]:
def get_accuracy(predictions, Y):
    return np.sum(predictions == Y)/Y.size

In [ ]:
def make_predictions(X, W1 ,b1, W2, b2):
    _, _, _, A2 = forward_propagation(X, W1, b1, W2, b2)
    predictions = get_predictions(A2)
    return predictions

In [ ]:
def gradient_descent(X, Y, alpha, iterations):
    size , m = X.shape

    W1, b1, W2, b2 = init_params(size)
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_propagation(X, W1, b1, W2, b2)
        dW1, db1, dW2, db2 = backward_propagation(X, Y, A1, A2, W2, Z1, m)

        W1, b1, W2, b2 = update_params(alpha, W1, b1, W2, b2, dW1, db1, dW2, db2)   

        if (i+1) % int(iterations/10) == 0:
            print(f"Iteration: {i+1} / {iterations}")
            prediction = get_predictions(A2)
            print(f'{get_accuracy(prediction, Y):.3%}')
    return W1, b1, W2, b2